In [2]:
import os
import random
import re

# Rename

In [3]:
def rename(fileDir, prefix, zCount, cont=False):
    
    files = os.listdir(fileDir)
    
    templateF = re.compile(f'{prefix}_{"[0-9]{"+str(zCount)+"}"}')
    templateC = re.compile(f'[0-9]{"{"+str(zCount)+"}"}')
    
    fileName = []
    contC = 0
    if(cont):
        for f in files:
            if(templateF.match(f) is None):
                fileName.append(f)
            else:
                c = templateC.search(f)
                if(int(c) > contC):
                    contC = int(c)
        
    else:
        fileName = files
    
    for i in range(len(fileName)):
        os.rename(os.path.join(fileDir, fileName), os.path.join(fileDir, f"{prefix}_{str.zfill((contC+i+1),zCount)}"))

# Generate Mask

In [4]:
import numpy as np
import cv2 as cv

import torch

In [5]:
import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)

In [6]:
def padCrop(img, targetSize):
    h, w, c = 0, 0, 1
        
    try:
        h, w, c = img.shape
    except:
        h, w = img.shape

    mH, mW, mC = 0, 0, 1
    try:
        mH, mW, mC = targetSize
    except:
        mH, mW = targetSize

    pz = np.zeros(targetSize, dtype=np.uint8)

    bH, bW, bC = mH, mW, mC

    if(bH > h):
        bH = h

    if(bW > w):
        bW = w

    if(bC == 1 and c == 3):
        img = cv.cvtColor(img, cv.COLOR_RGB2GRAY)
        c = 1
    elif(bC == 3 and c == 1):
        img = np.stack([img, img, img], axis=-1)
        c = 3

    pz[0:bH, 0:bW, :] = img[0:bH, 0:bW, :]
    
    return pz

In [10]:
#colorFormat : Dict -> k = Class {0,1,2,3,4} v = {[f, **low, **high]}
#Class 0 = Not found
#f means format :
#"B" = Binary (Assume only 1 obj)
#"R" = RGB
#"H" = HSV
#colorFormat = None -> All binary
def generateMasks(model, modelInpSize, fileDir, outDir, colorFormat=None, threshold=0.75):
    trans = get_tranform(False)
    
    files = [f for f in os.listdir(fileDir) if ((".PNG" in f) or (".png" in f) or (".JPG" in f) or (".jpg" in f))]
    
    #imgs = []
    #masks = []
    
    rgbMax = np.array([256,256,256], dtype=np.uint8)
    hsvMax = np.array([180,256,256], dtype=np.uint8)
    
    colorDiff = dict()
    if(colorFormat is not None):
        for k,v in colorFormat.items():
            if(v[0] != "B"):
                v1 = np.array(v1)
                v2 = np.array(v2)
                colorDiff[k] = v2-v1
    
    for f in files:
        #Read Img
        img = cv.imread(os.path.join(fileDir, f))
        img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
        
        H, W, C = img.shape
        
        #Pad and crop to match model Input size
        img = padCrop(img, modelInpSIze)
        
        with torch.no_grad():
            tImg, t = trans(img, dict())
            tImg = tImg.to('cuda')
            
            pred = model([tImg])[0]
            
            
            if(len(pred["masks"]) == 0):
                
                if(colorFormat is None or colorFormat[0][0] == "B"):
                    mask = np.zeros((H,W,1), dtype=np.uint8)
                else:
                    mask = np.zeros((H,W,3), dtype=np.uint8)
                    if(colorFormat[0][0] == "H"):
                        mask = cv.cvtColor(mask, cv.COLOR_RGB2HSV)
                
                cv.imwrite(os.path.join(fileDir,f"{f[:-4]}_Mask{f[-4:]}"), mask)
                continue
            
            cCount = dict()
            
            for k,v in colorFormat:
                cCount[k] = 0
            
            
            mask = np.zeros((H,W,3), dtype=np.uint8)
            scores = pred["scores"]
            labels = pred["labels"]
            i = 0
            
            while True:
                if(scores[i] > threshold):
                    
                    lab = labels[i]
                    
                    m = pred["masks"][i]
                    m = m.cpu().detach().numpy().squeeze()
                    r, m = cv.threshold(mask, 0.3, 255, cv.THRESH_BINARY)
                    
                    if(colorFormat is None or colorFormat[lab][0] == "B"):
                        if(i < 1):
                            mask = m
                            break
                    else:
                        flatC = np.zeros((H,W,3), dtype=np.uint8)
                        
                        cL = colorFormat[lab][1]
                        cH = colorFormat[lab][2]
                        cD = colorDiff[lab]
                        
                        curI = cCount[lab]
                        
                        color = np.array([0,0,0], dtype=np.uint8)
                        
                        for k in range(len(color)):
                            if(curI > abs(color[k])):
                                color[k] = abs(cD[k])
                                curI = curI - abs(cD[k])
                            else:
                                color[k] = curI
                                break
                                
                        color = np.multiply(color, np.sign(cD))
                        color = cL + color
                        
                        if(colorFormat[lab] == "H"):
                            color = np.mod(color, hsvMax)
                            color = cv.cvtColor(color, cv.COLOR_HSV2RGB)
                        else:
                            color = np.mod(color, rgbMax)
                        
                        flatC[:] = color
                        
                        maskC = cv.bitwise_and(flatC, flatC, mask=mask)
                        mask = cv.add(mask, maskC)
                        
                else:
                    break
                i+=1
                    
            
            cv.imwrite(os.path.join(fileDir,f"{f[:-4]}_Mask{f[-4:]}"), mask)


# Split

In [13]:
#Imgs -> List of file names
#Masks -> List of file names
def split(imgs, masks, p=(0.7, 0.3)):
    temp = list(zip(imgs,masks))
    random.shuffle(temp)
    imgs, masks = zip(*temp)
    
    n = math.floor(len(imgs)*p[0])
    return (imgs[:n], masks[:n]), (imgs[n:], masks[n:])

In [12]:
def transfer(srcDirImg, srcDirMask, dstDirImg, dstDirMask, dataset):
    imgsF, masksF = dataset
    
    for i in range(len(imgsF)):
        shutil.copy(os.path.join(srcDirImg, imgsF[i]), os.path.join(dstDirImg, imgsF[i]))
        shutil.copy(os.path.join(srcDirMask, masksF[i]), os.path.join(dstDirMask, masksF[i]))